In [48]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
!pip install contractions
import contractions


In [31]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df = pd.read_csv('/content/IPD_Reddit_Dataset.csv') # Adjust path if needed

In [33]:
print("Columns in dataset:", df.columns)
print(df.head())

Columns in dataset: Index(['text', 'depression'], dtype='object')
                                                text  depression
0                                         I love him           0
1  I got accepted into a Graduate program I wante...           0
2  our film production professor gave us our midt...           0
3  Just got done speaking with the inspector for ...           0
4  so i'm nonbinary and despite knowing that most...           0


In [49]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www\S+|@\S+|#\S+", "", text)  # Remove links/handles
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Remove emojis & special chars
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Keep only letters
    text = text.lower()
    text = re.sub(r'(.)\1{2,}', r'\1', text)  # Reduce repeated characters: "sooo" → "so"
    words = text.split()
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]
    return " ".join(words)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Apply cleaning
df['clean_text'] = df['text'].apply(clean_text)

In [52]:
# BERT Embeddings (new)
from sentence_transformers import SentenceTransformer

bert_model = SentenceTransformer('all-MiniLM-L6-v2')  # ~2x faster, decent accuracy


X = bert_model.encode(df['clean_text'].tolist(), show_progress_bar=True)


Batches:   0%|          | 0/149 [00:00<?, ?it/s]

In [53]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
df['sentiment'] = df['text'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

# Combine sentiment score with BERT embeddings
import numpy as np
X = np.hstack((X, df['sentiment'].values.reshape(-1, 1)))


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [54]:
print(df.columns)


Index(['text', 'depression', 'clean_text', 'sentiment'], dtype='object')


In [55]:
# 5. Labels
y = df['depression']  # target column

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


In [62]:
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBClassifier

param_dist = {
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

random_search = RandomizedSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    param_distributions=param_dist,
    n_iter=10,        # Tries only 10 random combos
    cv=3,
    verbose=2,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)






In [63]:


random_search.fit(X_train, y_train)
model = random_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:38:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [64]:
#grid_model.fit(X_train, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


KeyboardInterrupt: 

In [65]:
# Use best model
#model = grid_model.best_estimator_

# Predict
y_pred = model.predict(X_test)

#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)


In [66]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy of the model: {accuracy:.2f}")



Accuracy of the model: 0.92


In [67]:
import joblib
joblib.dump(model, "final_xgb_model.pkl")


['final_xgb_model.pkl']

In [68]:
import joblib
joblib.dump(model, "mental_health_model.pkl")


['mental_health_model.pkl']

In [71]:
bert_model.save("bert_encoder")



In [73]:
import shutil
shutil.make_archive("bert_encoder", 'zip', "bert_encoder")



'/content/bert_encoder.zip'

In [74]:
from google.colab import files
files.download("bert_encoder.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()


In [25]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [26]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy of the model: {accuracy:.2f}")


Accuracy of the model: 0.87
